# UBX Reader - u-blox Binary Format

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys

from datetime import datetime, timezone

import numpy as np

import unittest

from file_reader import FileReader

## Main Class

In [2]:
class UbxReader(FileReader):
    '''UBX file - u-blox Binary Format'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()
            
            # The memoryview will be used to access the buffer without intermediate copying
            self.bufferView = memoryview(self.buffer)

            if len(self.buffer) > 0:
                self.readData()


    def readData(self):
        '''Read NAV-PVT messages into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        while self.bufferPtr < fileSize:
            msgClass, msgId, payload = self.readMessage()
            
            if msgClass == 0x01 and msgId == 0x07:
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanBuffer += payload
                self.numRecords += 1
                       
        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        if len(leanBuffer) % self.numRecords != 0:
            raise ValueError('File appears to have varying message lengths')
        messageLen = len(leanBuffer) // self.numRecords

        dtype = self.getDatatype(messageLen)
        
        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def readMessage(self):
        '''Read UBX frame from buffer'''

        offset = self.bufferPtr

        syncChar1 = self.bufferView[self.bufferPtr]
        syncChar2 = self.bufferView[self.bufferPtr + 1]       
        msgClass = self.bufferView[self.bufferPtr + 2]
        msgId = self.bufferView[self.bufferPtr + 3]

        self.bufferPtr += 4

        if syncChar1 != 0xb5 or syncChar2 != 0x62:
            raise ValueError('Unrecognised sync chars - 0x{:02x} 0x{:02x}'.format(syncChar1, syncChar2))

        msgLength = self.readUnsigned16LE()       

        payload = self.readBytes(msgLength)

        expectedChecksum = self.readUnsigned16LE()

        # Evaluate checksum
        actualChecksum = self.calculateChecksum(self.bufferView[offset + 2 : offset + 6 + msgLength])
        if actualChecksum != expectedChecksum:
            raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        return msgClass, msgId, payload


    def calculateChecksum(self, message):
        '''Calculate Fletcher's checksum for the UBX message'''

        checksum_a = checksum_b = 0
        
        # Sum the message bytes
        for byte in message:
            checksum_a += byte
            checksum_b += checksum_a

        # Bit masking has been deferred until the end
        return (checksum_b & 0xff) << 8 | checksum_a & 0xff


    def getDatatype(self, messageLen):
        '''Get the datatype of the NAV-PVT payload'''

        # Standard 92-byte NAV-PVT payload
        dtype = [
            ('iTOW', '<u4'),
            ('year', '<u2'),
            ('month', 'u1'),
            ('day', 'u1'),
            ('hour', 'u1'),
            ('min', 'u1'),
            ('sec', 'u1'),
            ('valid', 'u1'),
            ('tAcc', '<u4'),
            ('nano', '<i4'),
            ('fixType', 'u1'),
            ('flags', 'u1'),
            ('flags2', 'u1'),
            ('numSV', 'u1'),
            ('lon', '<i4'),
            ('lat', '<i4'),
            ('height', '<i4'),
            ('hMSL', '<i4'),
            ('hAcc', '<u4'),
            ('vAcc', '<u4'),
            ('velN', '<i4'),
            ('velE', '<i4'),
            ('velD', '<i4'),
            ('gSpeed', '<i4'),
            ('headMot', '<i4'),
            ('sAcc', '<u4'),
            ('headAcc', '<u4'),
            ('pDOP', '<u2'),
            ('flags3', '<u2'),
            ('reserved', '<u4'),
            ('headVeh', '<i4'),
            ('magDec', '<i2'),
            ('magAcc', '<u2')
        ]

        # Forward compatibility / future-proofing in case NAV-PVT messages exceed 92 bytes in length
        for i in range(messageLen - 92):
            dtype.extend([
                ('unknown{:2}'.format(i), 'u1')
            ])

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['lat'] / 10000000
        self.data['lon'] = self.rawData['lon'] / 10000000

        # Convert elevation from cm to m
        self.data['ele'] = self.rawData['hMSL'] / 1000

        # Convert SOG from mm/s to m/s
        self.data['sog'] = self.rawData['gSpeed'] / 1000

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['headMot'] / 100000

        # Convert timestamp from ms to s
        #self.data['timestamp'] = self.rawData['utc_time'] / 1000

        # Convert date/time to regular timestamp (seconds)
        vfunc = np.vectorize(getDateTime)
        timestampSecs = vfunc(  self.rawData['year'], self.rawData['month'], self.rawData['day'],
                                self.rawData['hour'], self.rawData['min'], self.rawData['sec'])

        # Convert timestamps to milliseconds
        self.data['timestamp'] = timestampSecs + np.round(self.rawData['nano'] / 1000000000, 3)

        # Copy fix and satellites
        self.data['fix'] = self.rawData['fixType']
        self.data['sat'] = self.rawData['numSV']

        # Convert sAcc from mm/s to m/s
        self.data['sacc'] = self.rawData['sAcc'] / 1000

        # Convert hAcc and vAcc from mm to m
        self.data['hacc'] = self.rawData['hAcc'] / 1000
        self.data['vacc'] = self.rawData['vAcc'] / 1000
        
        # Convert cAcc to degrees
        self.data['cacc'] = self.rawData['headAcc'] / 100000

        # Convert PDOP from integer to decimal
        self.data['pdop'] = self.rawData['pDOP'] / 100
            

def getDateTime(year, month, day, hour, minute, second):
    '''Decode date/time'''

    return datetime(year, month, day, hour, minute, second, tzinfo=timezone.utc).timestamp()

## Unit Tests

In [3]:
class TestEspData(unittest.TestCase):
    '''Class to test ESP-GPS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(espReader.numRecords, 43444)

        for fieldName in espReader.data:
            self.assertEqual(espReader.data[fieldName].size, 43444)


    def testPdop(self):
        '''Test the positional dilution of precision is as expected'''

        self.assertEqual(espReader.data['pdop'].min(), 0.96)
        self.assertEqual(espReader.data['pdop'].max(), 1.88)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(espReader.data['sat'].min(), 9)
        self.assertEqual(espReader.data['sat'].max(), 18)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(espReader.data['timestamp'].min(), 1640864825.3)
        self.assertEqual(espReader.data['timestamp'].max(), 1640869169.6)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(espReader.data['lat'].min(), 51.6993591)
        self.assertEqual(espReader.data['lat'].max(), 51.7087559)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(espReader.data['lon'].min(), 4.0860906)
        self.assertEqual(espReader.data['lon'].max(), 4.0890658)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(espReader.data['ele'].min(), -3.159)
        self.assertEqual(espReader.data['ele'].max(), 11.978)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(espReader.data['sog'].min(), 0)
        self.assertEqual(espReader.data['sog'].max(), 18.169)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(espReader.data['cog'].min(), 0.01099)
        self.assertEqual(espReader.data['cog'].max(), 359.99525)


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(espReader.data['fix'].min(), 3)
        self.assertEqual(espReader.data['fix'].max(), 3)


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(espReader.data['sacc'].min(), 0.051)
        self.assertEqual(espReader.data['sacc'].max(), 0.771)


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(espReader.data['hacc'].min(), 0.169)
        self.assertEqual(espReader.data['hacc'].max(), 2.508)


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(espReader.data['vacc'].min(), 0.207)
        self.assertEqual(espReader.data['vacc'].max(), 2.582)


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(espReader.data['cacc'].min(), 0.23885)
        self.assertEqual(espReader.data['cacc'].max(), 44.99855)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.ubx')
    espReader = UbxReader(filename)
    espReader.load()
    
    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)
    
    print('All done!')

..............

All done!



----------------------------------------------------------------------
Ran 14 tests in 0.009s

OK
